## Loading images from directory as dataset

In [1]:
img_width, img_height = 100, 100
input_shape = (img_width, img_height, 1)
batch_size = 32
output_classes = 62

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1/255,
                                     shear_range=0.2, zoom_range=0.2)
data_generator = ImageDataGenerator(rescale=1/255)
test_generator = ImageDataGenerator(rescale=1/255)

train_ds = train_generator.flow_from_directory(
    directory="./train",
    target_size = (img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

validation_ds = data_generator.flow_from_directory(
    directory="./val",
    target_size = (img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

test_ds = test_generator.flow_from_directory(
    directory="./test",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

Found 2356 images belonging to 62 classes.
Found 682 images belonging to 62 classes.
Found 372 images belonging to 62 classes.


## Building CNN model

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout, Flatten

# Define the model
model = Sequential()

# Layer 1: Convolutional layer with 96 filters
model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', input_shape=input_shape, activation='relu'))

# Layer 2: Max pooling layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Layer 3: Zero padding layer
model.add(ZeroPadding2D(padding=(2,2)))

# Layer 4: Convolutional layer with 256 filters
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'))

# Layer 5: Max pooling layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Layer 6: Zero padding layer
model.add(ZeroPadding2D(padding=(1,1)))

# Layer 7: Convolutional layer with 384 filters
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))

# Layer 8: Zero padding layer
model.add(ZeroPadding2D(padding=(1,1)))

# Layer 9: Convolutional layer with 384 filters
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))

# Layer 10: Zero padding layer
model.add(ZeroPadding2D(padding=(1,1)))

# Layer 11: Convolutional layer with 256 filters
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))

# Layer 12: Max pooling layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Layer 13: Flatten layer
model.add(Flatten())

# Layer 14: Fully connected layer with 4096 units
model.add(Dense(4096, activation='relu'))

# Layer 15: Dropout layer
model.add(Dropout(0.5))

# Layer 16: Fully connected layer with 4096 units
model.add(Dense(4096, activation='relu'))

# Layer 17: Dropout layer
model.add(Dropout(0.5))

# Layer 18: Output layer with 62 layers
model.add(Dense(output_classes, activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 23, 23, 96)        11712     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 96)       0         
 )                                                               
                                                                 
 zero_padding2d (ZeroPadding  (None, 15, 15, 96)       0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 256)        0         
 2D)                                                             
                                                        

In [5]:
from keras import optimizers
model.compile(
    optimizers.RMSprop(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

## Train/Fit the model

In [6]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=6,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

In [7]:
epochs_range = 100
# Train the model
history = model.fit(train_ds, batch_size=32, 
                    epochs=epochs_range,callbacks= callback,
                    validation_data=validation_ds)

Epoch 1/100
74/74 [==============================] - 135s 2s/step - loss: 9.3804 - accuracy: 0.0140 - val_loss: 4.1272 - val_accuracy: 0.0161
Epoch 2/100
 2/74 [..............................] - ETA: 2:03 - loss: 4.1304 - accuracy: 0.0000e+00

## Plot the result

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(range(len(acc)), acc, label='Training Accuracy')
plt.plot(range(len(val_acc)), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(range(len(loss)), loss, label='Training Loss')
plt.plot(range(len(val_loss)), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Save the model

In [ ]:
# Save the model as model.h5
model.save('model_alexnet.h5')

In [ ]:
# Print the class indices 
print("Prediction Dict: ", train_ds.class_indices)